# Build Initial Solution Dataset

## Setup

In [22]:
# Setup and Imports
import json
import pandas as pd

## Load Datasets

In [23]:
prm800k_train = pd.read_csv("dataset/prm800k-train.csv")
prm800k_train

,labeler,timestamp,generation,is_quality_control_question,is_initial_screening_question,question,label
0,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-13T18:55:54.496450,NaN,False,False,{'problem': 'How many seconds are in 7.8 minut...,{'steps': [{'completions': [{'text': '7.8 minu...
1,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-17T16:56:51.323252,NaN,False,False,{'problem': 'How many positive two-digit integ...,"{'steps': [{'completions': [{'text': ""Let's ca..."
2,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-02T18:33:27.255302,NaN,False,False,{'problem': 'The fifth and eighth terms of a g...,{'steps': [{'completions': [{'text': 'So we ha...
3,d8aa7923-b970-45e1-9734-e4a7f6c4a7db,2022-07-17T16:56:53.345085,NaN,False,False,{'problem': 'Find the value of $x$ that satisf...,"{'steps': [{'completions': [{'text': ""Let's fi..."
4,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-17T16:59:27.072495,NaN,False,False,{'problem': 'What is the least three-digit who...,{'steps': [{'completions': [{'text': 'So we wa...
...,...,...,...,...,...,...,...
98726,340d89bc-f5b7-45e9-b272-909ba68ee363,2023-02-04T04:04:19.560459,9.0,False,False,{'problem': 'Find the largest possible value o...,{'steps': [{'completions': [{'text': 'This is ...
98727,340d89bc-f5b7-45e9-b272-909ba68ee363,2023-02-04T03:29:16.886449,9.0,False,False,"{'problem': 'Alicia has $n$ candies, where $n$...",{'steps': [{'completions': [{'text': 'I need t...
98728,2b794369-78c8-4e28-b3ba-eb0dd4a9ab2c,2023-02-03T23:46:18.766691,9.0,False,False,{'problem': 'Find the remainder when $$33818^2...,{'steps': [{'completions': [{'text': 'This pro...
98729,2b794369-78c8-4e28-b3ba-eb0dd4a9ab2c,2023-02-03T23:43:33.397886,9.0,False,False,{'problem': 'On the Cartesian plane in which e...,{'steps': [{'completions': [{'text': 'I need t...


In [24]:
prm800k_test = pd.read_csv("dataset/prm800k-test.csv")
prm800k_test

,labeler,timestamp,generation,is_quality_control_question,is_initial_screening_question,question,label
0,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-30T23:23:32.118856,NaN,False,False,{'problem': 'Three pencils and a jumbo eraser ...,"{'steps': [{'completions': [{'text': ""Let's ca..."
1,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-28T21:11:38.359626,NaN,False,False,"{'problem': 'Steve says to Jon, ""I am thinking...","{'steps': [{'completions': [{'text': ""Hey, you..."
2,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-28T21:14:58.004130,NaN,False,False,{'problem': 'Compute $58_9 - 18_9.$ Express yo...,{'steps': [{'completions': [{'text': 'I think ...
3,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-30T23:38:08.623029,NaN,False,False,{'problem': 'What is $\\sqrt{53}$ in simplest ...,"{'steps': [{'completions': [{'text': ""Let's fi..."
4,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-31T01:42:39.680550,NaN,False,False,{'problem': 'What is the sum of all of the mul...,{'steps': [{'completions': [{'text': 'I think ...
...,...,...,...,...,...,...,...
2863,423ec391-7def-48bd-821a-5e231333d6ce,2022-11-10T21:59:40.659723,6.0,False,False,{'problem': 'What is the integer value of $x$ ...,{'steps': [{'completions': [{'text': 'An arith...
2864,30f0b980-2587-4ec3-957a-0ca2bf958c40,2022-11-10T21:59:39.648690,6.0,False,False,{'problem': 'The binary number $10101001110_{2...,{'steps': [{'completions': [{'text': 'To conve...
2865,909003eb-d16e-49c8-82ce-7664180c66b8,2022-11-10T21:59:14.778080,6.0,False,False,{'problem': 'The binary number $10101001110_{2...,{'steps': [{'completions': [{'text': 'I know t...
2866,a2e09d3a-2da6-4094-89ca-e92125f6aefd,2022-11-10T21:58:33.614002,6.0,False,False,{'problem': 'Simplify $\\frac{(10r^3)(4r^6)}{8...,{'steps': [{'completions': [{'text': 'To simpl...


## Get Dataset Samples

*The Strategy*
Idea: Disquality Not Important Datas
Collect Dataset Sample with Slicing from Big to Small

1. Get Rows That Contains -1, 0, 1 (other Rows need dropped)

In [25]:
import json
import ast

def string_to_dict(input_string):
    input_string = input_string.strip()

    try:
        json_string = input_string.replace("'", '"')
        json_string = json_string.replace('True', 'true').replace('False', 'false').replace('None', 'null')
        return json.loads(json_string)
    except (json.JSONDecodeError, ValueError):
        pass
    
    try:
        return ast.literal_eval(input_string)
    except (ValueError, SyntaxError):
        pass
    
    try:
        if input_string.strip().startswith('{') and input_string.strip().endswith('}'):
            return eval(input_string)
    except:
        pass

    raise ValueError(f"Cannot parse string as dictionary: {input_string[:100]}...")

### 1. Get Rows That Contains -1, 0, 1 (other Rows need dropped)

In [26]:
def check_rating_completion(step):
    is_neg_1 = False
    is_zero = False
    is_pos_1 = False
    neg_1 = 0
    zero = 0
    pos_1 = 0
    
    
    for idx, completion in enumerate(step['completions']):
        rating = completion['rating']
        if rating == -1 and not is_neg_1:
            is_neg_1 = True
            neg_1 += 1
        elif rating == 0 and not is_zero:
            is_zero = True
            zero += 1
        elif rating == 1 and not is_pos_1:
            is_pos_1 = True
            pos_1 += 1

    human_completion = step['human_completion']

    rating_info = {
        "-1": neg_1,
        "0": zero,
        "1": pos_1,
        "human_completion": human_completion        
    }
    
    return is_neg_1, is_zero, is_pos_1, neg_1, zero, pos_1, rating_info

In [27]:
def get_row_contains_all_rating(prm800k_train):
    results = []
    
    for index, row in prm800k_train.iterrows():
        row_neg_1 = 0
        row_zero = 0
        row_pos_1 = 0
        total_steps_have_combination = 0
        total_steps = 0

        if len(results) == 1000:
            break

        try:
            data = string_to_dict(row['label'])
            finish_reason = data['finish_reason']
            rating_check = []
            
            # Process each step
            for step_no, step in enumerate(data['steps']):
                is_neg_1, is_zero, is_pos_1, neg_1, zero, pos_1, rating_info = check_rating_completion(step)
                row_neg_1 += neg_1
                row_zero += zero
                row_pos_1 += pos_1
                total_steps += 1
                rating_check.append(rating_info)
                
                # In one steps combined from 3 category
                if is_neg_1 and is_zero and is_pos_1: 
                    total_steps_have_combination += 1

            if row_neg_1 > 0 and row_zero > 0 and row_pos_1 > 0 and total_steps_have_combination > 0 and finish_reason == "solution":
                new_result = {
                    **row.to_dict(),
                    "row_neg_1": row_neg_1,
                    "row_zero": row_zero,
                    "row_pos_1": row_pos_1,
                    "total_steps_have_combination": total_steps_have_combination,
                    "total_steps": total_steps,
                    "finish_reason": finish_reason,
                    "rating_check": rating_check,
                }
                results.append(new_result)
        
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            continue
    
    # Create DataFrame from results
    results_df = pd.DataFrame(results)
    return results_df

In [28]:
df_experiment_preparation = get_row_contains_all_rating(prm800k_train)
df_experiment_preparation = df_experiment_preparation.sort_values('total_steps_have_combination', ascending=False)
df_experiment_preparation.to_csv('dataset/prm800k-experiment-preparation.csv', index=False)

Error processing row 133: 'NoneType' object is not iterable
Error processing row 226: 'NoneType' object is not iterable
Error processing row 239: 'NoneType' object is not iterable
Error processing row 257: 'NoneType' object is not iterable
Error processing row 258: 'NoneType' object is not iterable
Error processing row 594: 'NoneType' object is not iterable
Error processing row 595: 'NoneType' object is not iterable
Error processing row 596: 'NoneType' object is not iterable
Error processing row 597: 'NoneType' object is not iterable
Error processing row 598: 'NoneType' object is not iterable
Error processing row 650: 'NoneType' object is not iterable
Error processing row 651: 'NoneType' object is not iterable
Error processing row 652: 'NoneType' object is not iterable
Error processing row 724: 'NoneType' object is not iterable
Error processing row 761: 'NoneType' object is not iterable
Error processing row 762: 'NoneType' object is not iterable
Error processing row 789: 'NoneType' obj

In [29]:
df_experiment_preparation

,labeler,timestamp,generation,is_quality_control_question,is_initial_screening_question,question,label,row_neg_1,row_zero,row_pos_1,total_steps_have_combination,total_steps,finish_reason,rating_check
368,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-02T18:19:39.244785,NaN,False,False,"{'problem': 'Define the sequence $a_1, a_2, a_...",{'steps': [{'completions': [{'text': 'What exa...,34,29,48,16,64,solution,"[{'-1': 1, '0': 1, '1': 1, 'human_completion':..."
198,d8aa7923-b970-45e1-9734-e4a7f6c4a7db,2022-07-16T06:04:01.122899,NaN,False,False,"{'problem': 'Harold, Tanya, and Ulysses paint ...","{'steps': [{'completions': [{'text': ""Let's sa...",32,27,45,15,60,solution,"[{'-1': 1, '0': 1, '1': 1, 'human_completion':..."
142,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-15T16:01:50.243286,NaN,False,False,{'problem': 'What is the sum of all the positi...,"{'steps': [{'completions': [{'text': ""Let's ca...",27,22,51,12,62,solution,"[{'-1': 0, '0': 0, '1': 1, 'human_completion':..."
265,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-12T20:22:22.292498,NaN,False,False,{'problem': 'The first digit of a string of 20...,"{'steps': [{'completions': [{'text': ""Let's fi...",45,20,46,12,68,solution,"[{'-1': 0, '0': 0, '1': 1, 'human_completion':..."
337,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-07T14:48:59.553869,NaN,False,False,{'problem': 'Triangle $ABC$ has positive integ...,"{'steps': [{'completions': [{'text': ""Hey, whe...",27,25,40,12,53,solution,"[{'-1': 1, '0': 1, '1': 1, 'human_completion':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-23T17:20:48.793671,NaN,False,False,{'problem': 'What are the last two digits in t...,{'steps': [{'completions': [{'text': '1! is ju...,10,3,30,1,35,solution,"[{'-1': 0, '0': 0, '1': 1, 'human_completion':..."
213,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-27T17:24:41.556043,NaN,False,False,{'problem': 'Two transformations are applied t...,{'steps': [{'completions': [{'text': 'We can t...,3,3,14,1,16,solution,"[{'-1': 0, '0': 1, '1': 0, 'human_completion':..."
217,e90a38f3-3135-4465-87af-3e6322e3d772,2022-06-30T21:40:52.528252,NaN,False,False,{'problem': 'Solve for $x$: $x+2x = 400-(3x+4x...,{'steps': [{'completions': [{'text': 'So we ha...,1,1,7,1,7,solution,"[{'-1': 1, '0': 1, '1': 1, 'human_completion':..."
218,a5d23a92-ab95-4274-af7a-dd1315e81403,2022-07-19T15:43:48.450161,NaN,False,False,"{'problem': 'If $4^6=8^n$, what is $n$?', 'gro...",{'steps': [{'completions': [{'text': 'Since $8...,1,1,5,1,5,solution,"[{'-1': 1, '0': 1, '1': 1, 'human_completion':..."


### Versioning:
1. v1.0: row_neg_1 > 0 and row_zero > 0 and row_pos_1 > 0: 37,543 rows
2. v2.0: row_neg_1 > 0 and row_zero > 0 and row_pos_1 > 0 and total_steps_have_combination > 0: 16,535 rows
3. v3.0: row_neg_1 > 1 and row_zero > 1 and row_pos_1 > 1 and total_steps_have_combination > 0: 385 rows
4. Sorting from most total_steps_have_combination
5. Collect 1000 rows
6. Collect only finish_reason == "solution"
7. Add rating check and info to check rating in every completions

## Build Fair Datasets

In [32]:
df_experiment_preparation = pd.read_csv("dataset/prm800k-experiment-preparation.csv")
df_experiment_preparation

,labeler,timestamp,generation,is_quality_control_question,is_initial_screening_question,question,label,row_neg_1,row_zero,row_pos_1,total_steps_have_combination,total_steps,finish_reason,rating_check
0,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-02T18:19:39.244785,NaN,False,False,"{'problem': 'Define the sequence $a_1, a_2, a_...",{'steps': [{'completions': [{'text': 'What exa...,34,29,48,16,64,solution,"[{'-1': 1, '0': 1, '1': 1, 'human_completion':..."
1,d8aa7923-b970-45e1-9734-e4a7f6c4a7db,2022-07-16T06:04:01.122899,NaN,False,False,"{'problem': 'Harold, Tanya, and Ulysses paint ...","{'steps': [{'completions': [{'text': ""Let's sa...",32,27,45,15,60,solution,"[{'-1': 1, '0': 1, '1': 1, 'human_completion':..."
2,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-15T16:01:50.243286,NaN,False,False,{'problem': 'What is the sum of all the positi...,"{'steps': [{'completions': [{'text': ""Let's ca...",27,22,51,12,62,solution,"[{'-1': 0, '0': 0, '1': 1, 'human_completion':..."
3,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-12T20:22:22.292498,NaN,False,False,{'problem': 'The first digit of a string of 20...,"{'steps': [{'completions': [{'text': ""Let's fi...",45,20,46,12,68,solution,"[{'-1': 0, '0': 0, '1': 1, 'human_completion':..."
4,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-07T14:48:59.553869,NaN,False,False,{'problem': 'Triangle $ABC$ has positive integ...,"{'steps': [{'completions': [{'text': ""Hey, whe...",27,25,40,12,53,solution,"[{'-1': 1, '0': 1, '1': 1, 'human_completion':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-23T17:20:48.793671,NaN,False,False,{'problem': 'What are the last two digits in t...,{'steps': [{'completions': [{'text': '1! is ju...,10,3,30,1,35,solution,"[{'-1': 0, '0': 0, '1': 1, 'human_completion':..."
425,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-27T17:24:41.556043,NaN,False,False,{'problem': 'Two transformations are applied t...,{'steps': [{'completions': [{'text': 'We can t...,3,3,14,1,16,solution,"[{'-1': 0, '0': 1, '1': 0, 'human_completion':..."
426,e90a38f3-3135-4465-87af-3e6322e3d772,2022-06-30T21:40:52.528252,NaN,False,False,{'problem': 'Solve for $x$: $x+2x = 400-(3x+4x...,{'steps': [{'completions': [{'text': 'So we ha...,1,1,7,1,7,solution,"[{'-1': 1, '0': 1, '1': 1, 'human_completion':..."
427,a5d23a92-ab95-4274-af7a-dd1315e81403,2022-07-19T15:43:48.450161,NaN,False,False,"{'problem': 'If $4^6=8^n$, what is $n$?', 'gro...",{'steps': [{'completions': [{'text': 'Since $8...,1,1,5,1,5,solution,"[{'-1': 1, '0': 1, '1': 1, 'human_completion':..."


### Select Only finish_reason == "solution"